Необходимые библиотеки

In [1]:
import numpy as np
import pandas as pd
import xlearn as xl
from tqdm.notebook import tqdm
from sklearn.metrics import log_loss
from sklearn.model_selection import TimeSeriesSplit

Загрузка данных, убираю неиспользуемые колонки

In [2]:
data = pd.read_csv('../../data/data.csv')
unused = 'banner_id0, banner_id1, rate0, rate1, g0, g1, coeff_sum0, coeff_sum1'.split(', ')
data = data.drop(columns=unused)
data.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,1,1


Выбираю последний день датасета и разделяю по нему

In [3]:
data['date_time'].max()

'2021-10-02 23:59:59.000000'

In [4]:
last_day = '2021-10-02'
data = data.sort_values(by='date_time')
splitting_mask = data['date_time'].str.startswith(last_day)

Feature engineering почти такой же, как в первой работе, удаляем impressions, которые всегда 1, выделяем из даты час и день недели.

In [14]:
def feature_engineering(data: pd.DataFrame) -> pd.DataFrame:
    # удаляем столбец impressions
    data = data.drop(columns=['impressions'])
    
    # выделяем из даты часы
    dates = pd.to_datetime(data['date_time'])
    data['hour'] = dates.dt.hour
    data['weekday'] = dates.dt.dayofweek
    data = data.drop(columns=['date_time'])
    
    # категоризуем oaid_hash, иначе xlearn умирает от слишком большого числа фич
    data['oaid_hash'] = data['oaid_hash'].astype('category').cat.codes

    return data

Запуск feature engineering и разделение датасета

In [15]:
data_fe = feature_engineering(data)
data_train, data_test = data_fe[~splitting_mask], data_fe[splitting_mask]

In [16]:
data_fe.head()

,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,clicks,hour,weekday
1390198,30,596,3693539,0,0,7,0,0,2
5041415,41,29,1293475,1,3,0,0,0,6
1442602,1,188,5233288,2,2,15,0,0,6
7232498,17,52,1292247,2,2,5,0,0,6
14938691,47,73,2947801,1,4,13,0,0,6


Выбираем для каждого столбца наибольшие значения

In [17]:
max_values = dict()
for col in data_fe.columns:
    max_values[col] = data_fe[col].max()

Составляем файлы для тренировки FFM

In [18]:
def create_field_string(data_row: pd.DataFrame, max_values: dict, features: list, field_no: int):
    current_max = 0
    field = []
    for feature in features:
        field.append(f"{field_no}:{current_max + data_row[feature]}:{1}")
        current_max += max_values[feature] + 1
    return (' ').join(field)
        

def create_libffm_file(data: pd.DataFrame, max_values: dict, filename: str):
    lines = []
    for i in tqdm(data.index):
        row = data.loc[i]
        label = str(row['clicks'])
        user_features = ['os_id', 'country_id']
        user_field = create_field_string(row, max_values, user_features, 0)
        ad_features = ['banner_id', 'zone_id', 'campaign_clicks']
        ad_field = create_field_string(row, max_values, ad_features, 1)
        time_features = ['hour', 'weekday']
        time_field = create_field_string(row, max_values, time_features, 2)
        lines.append(f"{label} {user_field} {ad_field} {time_field}")
    with open(filename, 'w') as file:
        file.write('\n'.join(lines))

In [19]:
train_file, test_file = 'train.txt', 'test.txt'
create_libffm_file(data_train, max_values, train_file)
create_libffm_file(data_test, max_values, test_file)

Теперь файлы собраны, строки в них находятся в порядке времени, поэтому валидационные фолды можно будет собирать сразу из них

In [25]:
def create_model(train_file: str, model_file: str, k: int):
    """
    Создаёт ffm модель для заданного k
    """
    ffm = xl.create_ffm()
    ffm.setTrain(train_file)
    param = {'task': 'binary', 'lr': 0.02, 'lambda': 0.02, 'k': k, 'metric': 'f1'}
    ffm.fit(param, model_path=model_file)
    return ffm


def evaluate_model(model, model_file, test_file):
    """
    Вычисляет log loss для модели на заданном тесте
    """
    model.setTest(test_file)
    model.setSigmoid()
    model.predict(model_file, './predictions.txt')
    with open(test_file) as file:
        labels = [int(line[0]) for line in file.read().split('\n')]
    with open('./predictions.txt') as file:
        pred = [float(p) for p in file.read().strip().split('\n')]
    return log_loss(labels, pred)


def baseline_score(test_file: str) -> float:
    """
    Log loss предсказания средним
    """
    with open(test_file) as file:
        labels = [int(line[0]) for line in file.read().split('\n')]
    return log_loss(labels, [np.mean(labels)] * len(labels))


def create_folds(train_file: str):
    """
    Создаёт фолды для валидации по времени
    """
    time_series = TimeSeriesSplit(n_splits=3)
    folds = []
    with open(train_file) as file:
        data = np.array(file.read().split('\n'))
        for i, (train_index, test_index) in enumerate(time_series.split(data)):
            with open(f'./val_train_{i}.txt', 'w') as f:
                f.write('\n'.join(data[train_index]))
            with open(f'./val_test_{i}.txt', 'w') as f:
                f.write('\n'.join(data[test_index]))
            folds.append((f'./val_train_{i}.txt', f'./val_test_{i}.txt'))
    return folds


def cv(folds):
    """
    Валидация по времени из собранных фолдов
    """
    best_loss, best_k = np.infty, None
    for k in [2, 5, 10, 15]:
        scores = []
        for train, test in folds:
            model = create_model(train, './val_model.out', k)
            scores.append(evaluate_model(model, './val_model.out', test))
        loss = np.mean(scores)
        if loss < best_loss:
            best_loss = loss
            best_k = k
    return create_model('./train.txt', './model.out', best_k), best_k

Посчитаем метрику для бейзлайна

In [21]:
baseline_score('./test.txt')

0.15303289904918682

Проведём валидацию по времени и выберем лучшее k (из рассмотренных)

In [22]:
folds = create_folds('./train.txt')

In [23]:
best_model, best_k = cv(folds)
best_k

15

In [26]:
evaluate_model(best_model, './model.out', './test.txt')

0.1467454444693803

Лучшим k оказалось 15, результат модели лучше бейзлайна и сопоставим с результатом первой лабораторной работы